In [10]:
import openai
import os
import time
import json
import requests
# print openai version

def talk_to_gpt(s):
    return openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            {"role": "system", "content": "You are a helpful assistant. You can search our database by doing search(query here)>"},
            {"role": "user", "content": s},
        ]
    )['choices'][0]['message']['content']

# text to speech
def speak(text):
    from win32com.client import Dispatch
    speak = Dispatch("SAPI.SpVoice")
    speak.Speak(text)

speak(talk_to_gpt("hi gpt"))

# listen for 5 seconds feed it into gpt and speak the response
def callback(recognizer, audio):
    try:
        # this should stop earlier as the environment is very noisy
        speech_as_text = recognizer.recognize_google(audio, language="en-US", show_all=False)
        print(speech_as_text)
        if "hey google" in speech_as_text.lower():
            # Perform your action here
            print("Performing action...")
            speak(talk_to_gpt(speech_as_text))
    except sr.UnknownValueError:
        pass
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))

KeyboardInterrupt: 

In [ ]:
# spokestack
!pip install spokestack --user
from spokestack.io.pyaudio import PyAudioInput
from spokestack.activation_timeout import ActivationTimeout
from spokestack.io.pyaudio import PyAudioInput
from spokestack.pipeline import SpeechPipeline
from spokestack.vad.webrtc import VoiceActivityDetector
from spokestack.wakeword.tflite import WakewordTrigger
from spokestack.asr.spokestack.speech_recognizer import SpeechRecognizer

mic = PyAudioInput()
vad = VoiceActivityDetector()
wake = WakewordTrigger("path_to_tflite_model")
asr = SpeechRecognizer("spokestack_id", "spokestack_secret")
timeout = ActivationTimeout()


pipeline = SpeechPipeline(mic, [vad, wake, asr, timeout])
pipeline.run()

^C


ModuleNotFoundError: No module named 'spokestack'

In [ ]:
import gradio as gr
import openai, subprocess
import pyaudio
import wave
messages = [{"role": "system", "content": 'You are a therapist. Respond to all input in 25 words or less.'}]

def output_file_recording():


    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 44100
    RECORD_SECONDS = 5
    WAVE_OUTPUT_FILENAME = "output.wav"

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("* recording")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("* done recording")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
def listen_for_5_seconds():
    global messages
    output_file_recording()
    # listen for 5 seconds using the microphone with pyaudio
    audio_file= open('output.wav', "rb")

    transcript = openai.Audio.transcribe("whisper-1", audio_file)
    print(transcript)
    messages.append({"role": "user", "content": transcript["text"]})

    response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)
    print(response)
    system_message = response["choices"][0]["message"]
    messages.append(system_message)

    speak(system_message["content"])
    chat_transcript = ""
    for message in messages:
        if message['role'] != 'system':
            chat_transcript += message['role'] + ": " + message['content'] + "\n\n"

    return chat_transcript


In [ ]:
import struct
import pyaudio
import pvporcupine

porcupine = None
pa = None
audio_stream = None

try:
    porcupine = pvporcupine.create(keywords=["computer", "jarvis"], access_key="key here")

    pa = pyaudio.PyAudio()

    audio_stream = pa.open(
                    rate=porcupine.sample_rate,
                    channels=1,
                    format=pyaudio.paInt16,
                    input=True,
                    frames_per_buffer=porcupine.frame_length)

    while True:
        pcm = audio_stream.read(porcupine.frame_length)
        pcm = struct.unpack_from("h" * porcupine.frame_length, pcm)

        keyword_index = porcupine.process(pcm)

        if keyword_index >= 0:
            listen_for_5_seconds()
            speak("Computer online")
except Exception as e:
    print("somethignwent wrong")
    print(e)

* recording
* done recording
{
  "text": "Hello GPT"
}
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Hello! How can I assist you today?",
        "role": "assistant"
      }
    }
  ],
  "created": 1677781619,
  "id": "chatcmpl-6phvnpFYmLBVECEZR0wYC8QzKicmX",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 11,
    "prompt_tokens": 31,
    "total_tokens": 42
  }
}
* recording
* done recording
{
  "text": "Let's. Watch good."
}
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "I'm not sure what you mean. Can you please provide more information or clarify your request?",
        "role": "assistant"
      }
    }
  ],
  "created": 1677781639,
  "id": "chatcmpl-6phw7q7sZcT8Mgy4XZ6DWCiJvcVun",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 21,
    "prompt_token

KeyboardInterrupt: 